In [0]:
# Day 0
# Import Kaggle CLI
!pip install kaggle

In [0]:
# Set Kaggle credentials
import os

os.environ["KAGGLE_USERNAME"] = "rajputdevyansh"
os.environ["KAGGLE_KEY"] = "KGAT_5c14f6f7ee747f3d344153b19539599c"

print("Kaggle credentials configured!")

In [0]:
# Created Database Schema
spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

In [0]:
# Created Volume for Data Storage
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

Downloading Dataset from Kaggle

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Extracting Downloaded Dataset

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

Clean the Zip File

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

In [0]:
# Restart Python Environment
%restart_python

In [0]:
# Loading November 2019 Data
df_n = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")

In [0]:
# Loading October 2019 Data
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

In [0]:
# Checking Row Counts and Schema
print(f"October 2019 - Total Events: {df.count():,}")
print("SCHEMA:")
df.printSchema()

print(f"November  2019 - Total Events: {df_n.count():,}")
print("SCHEMA:")
df.printSchema()

In [0]:
# fixing Schema inference
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, LongType, DoubleType

schema = StructType([
    StructField("event_time", TimestampType(), True),
    StructField("event_type", StringType(), True),
    StructField("product_id", LongType(), True),
    StructField("category_id", LongType(), True),
    StructField("category_code", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("user_id", LongType(), True),
    StructField("user_session", StringType(), True)
])

df = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",
    header=True,
    schema=schema
)

df_n = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",
    header=True,
    schema=schema
)

In [0]:
# Verifying Row Counts and Schema Again
print(f"October 2019 - Total Events: {df.count():,}")
print("SCHEMA:")
df.printSchema()

print(f"November  2019 - Total Events: {df_n.count():,}")
print("SCHEMA:")
df.printSchema()

In [0]:
# Sample October Preview
df.show(5, truncate=False)

In [0]:
# Sample November Preview
df_n.show(5, truncate=False)

In [0]:
# Day 1
# Viewing Distinct event types
df.select("event_type").distinct().show()

In [0]:
# Grouping by Event distribution
df.groupBy("event_type").count().orderBy("count", ascending=False).show()

In [0]:
# Day 2
# basic slow(select) query to view to top brands
df.select("event_type", "brand", "price").show(10)

In [0]:
# Filtering by price
df.filter(df.price > 100).show(10)

In [0]:
# Geting count for products above 1000
premium_count = df.filter(df.price > 1000).count()
print("Products above 1000:", premium_count)

In [0]:
# Grouping by event type
df.groupBy("event_type").count().show()

In [0]:
# grouping by brand
df.groupBy("brand").count().orderBy("count", ascending=False).show(10)

In [0]:
# Ordering by brand and price
df.select("brand", "price").orderBy(df.price.desc()).show(10)

In [0]:
# Creating a temp view
df.createOrReplaceTempView("events_oct")

Using Notebook Magic Commands(%sql)

In [0]:
%sql
SELECT event_type, COUNT(*) as cnt
FROM events_oct
GROUP BY event_type
ORDER BY cnt DESC

Using Notebook Magic Commands(%python)

In [0]:
%python
df.select("brand", "price").orderBy(df.price.desc()).show(5)

Using Notebook Magic Commands(%fs File Navigation)

In [0]:
%fs ls /Volumes/workspace/ecommerce/ecommerce_data

In [0]:
# Creating a small aggregated DataFrame for top brands in oct
top_brands = (
    df.groupBy("brand")
      .count()
      .orderBy("count", ascending=False)
      .limit(10)
)

top_brands.show()

In [0]:
# Writing the top brands to a csv
top_brands.write.mode("overwrite").csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/top_brands_oct",
    header=True
)

Verifying in the results in storage using %fs

In [0]:
%fs ls /Volumes/workspace/ecommerce/ecommerce_data/top_brands_oct